## Calculate terminated results exiobase v3.3.50005  exc. iLUC

from 'MR_HIOT_2011_v3.3.50005.xlsx'
and from 'MR_HUSE_2011_v3.2.50005_final_EXT.xlsx'



In [1]:
cd /Users/marie/Desktop

/Users/marie/Desktop


First, the xls file is turned into a CSV file to facilitate calculations. 
The xsl file can also be registered as a csv file.
Doing it manually is faster but executing the command below allows for multi-tasking

In [2]:
### optional
import pandas as pd
import csv

data_xls = pd.read_excel('MR_HIOT_2011_v3.3.50005.xlsx', 'HIOT', index_col=None)
data_xls.to_csv('MR_HIOT.csv', encoding='utf-8')

We create 3 csv files out of the MR-HUSE.csv file:
- Zn_tonorm.csv file (inputs) per activity that we will need to normalize
- Bn_tonorm.csv file (raw, occupation and emissions) per activity that we will need to normalize
- norm.csv file with the total consumption per activity
- final.csv with the final consumption activities

In [8]:
import pandas as pd
import csv

outfile1 ="/Users/marie/Desktop/Zn_tonorm_pd.csv"
source = pd.read_csv('MR_HIOT.csv', index_col = None, header = None, low_memory = False)

Zn_tonorm = source.iloc[7:7879, 6:7878]
Zn_tonorm.to_csv(outfile1, header = None)

In [4]:
import pandas as pd
import csv

outfile2 ="/Users/marie/Desktop/norm_pd.csv"
source = pd.read_csv('MR_HIOT.csv', index_col = None, header = None, low_memory = False)

norm = source.iloc[1:2, 6:7878]
norm.to_csv(outfile2, header = None)

In [61]:
import pandas as pd
import csv

data_xls = pd.read_excel('MR_HUSE_2011_v3.2.50005_final_EXT.xlsx', 'Emiss_act', index_col=None)
data_xls.to_csv('Emiss_act.csv', encoding='utf-8')

In [67]:
outfile3 ="/Users/marie/Desktop/Bn_tonorm_pd.csv"
source1 = pd.read_csv('resource_act.csv', index_col = None, header = None, low_memory = False)

Bn_tonorm = source1.iloc[6:41, 6:7878]

Bn_tonorm.to_csv(outfile3, header = None)

In [9]:
import pandas as pd
import csv

data_xls = pd.read_excel('MR_HIOT_2011_v3.3.50005.xlsx', 'FD', index_col=None)
data_xls.to_csv('FD.csv', encoding='utf-8')

In [7]:
import pandas as pd
import csv

outfile4 ="/Users/marie/Desktop/final_pd.csv"
source2 = pd.read_csv('FD.csv', index_col = None, header = None, low_memory = False)

final = source2.iloc[6:7879, 6:294]
final.to_csv(outfile4, header = None)

In [63]:
import pandas as pd
import csv

data_xls = pd.read_excel('MR_HUSE_2011_v3.2.50005_final_EXT.xlsx', 'Emiss_FD', index_col=None)
data_xls.to_csv('emiss_FD.csv', encoding='utf-8')

In [64]:
outfile5 ="/Users/marie/Desktop/final_emiss_pd.csv"
source3 = pd.read_csv('emiss_FD.csv', index_col = None, header = None, low_memory = False)

final_emiss = source3.iloc[6:70, 6:294]

final_emiss.to_csv(outfile5, header = None)

Remove the index row from pd frames

In [69]:
import csv
def remove_first_col(source, result):
    with open(source,"r") as source:
        rdr = csv.reader(source)
        with open (result, "w") as result:
            wtr = csv.writer(result)
            for r in rdr:
                del r[0]
                wtr.writerow(r)

In [70]:
cd /Users/marie/Desktop

/Users/marie/Desktop


In [71]:
remove_first_col("Bn_tonorm_pd.csv", "Bn_tonorm.csv")

In [68]:
### optional
import os

os.getcwd()
os.chdir('/Users/marie/Desktop')
os.remove("Bn_tonorm_pd.csv")
os.remove("final_emiss_pd.csv")

Replace 0s with 1s in norm.csv (matrices can't be divided by 0)

In [19]:
def replace_0with1(source, result):
    with open(source,"r") as source:
        rdr = csv.reader(source)
        with open (result, "w") as result:
            wtr = csv.writer(result)
            for row in rdr:
                row = [x.replace('0', '1') if x == '0' else x for x in row]
                wtr.writerow(row)

In [20]:
replace_0with1("norm.csv", "norm1.csv")

We need to read all the CSV files as matrices to make operations with the numpy package:
- Zn_tonorm.csv as matrice
- norm1.csv as vector
- Bn_tonorm.csv as a matrice

In [2]:
import csv
import numpy as np
with open("Zn_tonorm.csv",'r') as dest_f:
    data_iter = csv.reader(dest_f, 
                           delimiter = ',', 
                           quotechar = '"')
    data = [data for data in data_iter]
Zn_tonorm = np.array(list(data)).astype('float')

In [3]:
with open('norm1.csv','r') as dest_f:
    data_iter = csv.reader(dest_f, 
                           delimiter = ',', 
                           quotechar = '"')
    data = [data for data in data_iter]
nor = np.asarray(data, dtype='float')

In [4]:
with open("Bn_tonorm.csv",'r') as dest_f:
    data_iter = csv.reader(dest_f, 
                           delimiter = ',', 
                           quotechar = '"')
    data = [data for data in data_iter]
Bn_tonorm = np.array(list(data)).astype('float')

In [5]:
with open("final.csv",'r') as dest_f:
    data_iter = csv.reader(dest_f, 
                           delimiter = ',', 
                           quotechar = '"')
    data = [data for data in data_iter]
f_cons = np.array(list(data)).astype('float')

In [6]:
with open("final_emiss.csv",'r') as dest_f:
    data_iter = csv.reader(dest_f, 
                           delimiter = ',', 
                           quotechar = '"')
    data = [data for data in data_iter]
f_em = np.array(list(data)).astype('float')

To obtain Zn and Bn, Zn_tonorm and Bn_tonorm needs to be didvided by the norm vector.

In [7]:
Zn = Zn_tonorm/nor

In [8]:
Bn = Bn_tonorm/nor

We create the identity matrice

In [9]:
identity = np.matrix(np.identity(7872), copy=False)

In [10]:
An = identity-Zn

In [11]:
S = np.linalg.inv(An)

In [12]:
BLCI = Bn*S

In [13]:
from io import StringIO
import numpy as np
s=StringIO()
np.savetxt('BLCI.csv', BLCI, fmt='%.10f', delimiter=',', newline="\n")

In [14]:
F = BLCI*f_cons

In [15]:
from io import StringIO
import numpy as np
s=StringIO()
np.savetxt('F.csv', F, fmt='%.10f', delimiter=',', newline="\n")

In [86]:
F2 = F+f_em

In [87]:
from io import StringIO
import numpy as np
s=StringIO()
np.savetxt('F2.csv', F2, fmt='%.10f', delimiter=',', newline="\n")